In [1]:
import pandas as pd
import scipy.stats

# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

## Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

$$F_{c}(C,T) = \begin{cases} \frac{T}{C}, T&gt;C \\ -\frac{C}{T}, T&lt;C \end{cases}$$
где C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

* для групп normal (control) и early neoplasia (treatment)
* для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [55]:
def write_answer(name, data):
    with open(f"{name}.txt", "w") as fout:
        check = data.split()
        if check:
            fout.write(" ".join([num for num in check]))
        else:
            fout.write(str(data))

In [2]:
data_expr = pd.read_csv('../../gene_high_throughput_sequencing.csv')

In [3]:
data_expr.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [4]:
data_expr.describe()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,...,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,2.463922,7.100958,19.056151,15.618688,23.537120,11.303466,8.921396,63.270147,53.906324,70.678573,...,5.563444,2.192029,1.967424,2.181360,1.729308,1.980733,16.834075,1.825827,2.287380,1.756827
std,1.413687,4.164703,9.287753,4.664146,4.475294,4.055122,3.270713,13.792140,45.635498,21.326237,...,2.389789,2.454218,2.047129,2.542652,1.507149,2.230157,7.126802,1.902716,3.185571,1.623151
min,0.833898,1.231043,4.941211,6.395270,8.520713,2.066576,1.364917,42.544976,24.616767,42.631422,...,1.144240,0.853957,0.853957,0.853957,0.833898,0.833898,4.675683,0.833898,0.853957,0.833898
25%,1.284642,3.892403,13.423946,12.574596,21.151624,8.712898,6.213396,53.093627,34.009687,54.202100,...,3.805875,1.042783,1.011350,1.011350,1.003337,1.003337,11.186633,1.003337,1.011350,1.003337
50%,2.104677,6.315551,16.734855,14.586150,23.477006,11.172590,8.703397,63.230911,39.554954,65.510651,...,5.617824,1.339507,1.273861,1.298543,1.252527,1.252527,16.514389,1.252527,1.273861,1.252527
75%,3.345067,8.916399,21.884325,17.695678,26.471909,13.568625,11.139240,69.880705,50.295756,79.427207,...,7.100942,1.815754,1.647255,1.677971,1.625364,1.611299,21.849385,1.607345,1.647255,1.611299
max,7.364879,20.006038,60.584449,29.659104,34.110743,27.441093,17.121366,132.144503,327.590426,128.606260,...,11.227770,11.913855,10.883310,11.692697,10.392539,10.640754,49.295538,9.919132,17.278985,9.333904


In [5]:
data_expr.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [12]:
gene_for_comparison = filter(lambda x: x not in ['Patient_id', 'Diagnosis'], data_expr.columns.tolist())

In [6]:
normal_group = data_expr[data_expr.Diagnosis == 'normal']
early_neoplasia_group = data_expr[data_expr.Diagnosis == 'early neoplasia']
cancer_group = data_expr[data_expr.Diagnosis == 'cancer']

In [13]:
# сравнение групп
normal_and_early_neoplasia = {}
early_neoplasia_and_cancer = {}

for gene in gene_for_comparison:
    normal_and_early_neoplasia[gene] = scipy.stats.ttest_ind(normal_group[gene], early_neoplasia_group[gene], equal_var=False).pvalue
    early_neoplasia_and_cancer[gene] = scipy.stats.ttest_ind(early_neoplasia_group[gene], cancer_group[gene], equal_var=False).pvalue

In [15]:
normal_and_early_neoplasia_df = pd.DataFrame.from_dict(normal_and_early_neoplasia, orient='index')
normal_and_early_neoplasia_df.columns = ['normal_and_early_neoplasia_p-value']

early_neoplasia_and_cancer_df = pd.DataFrame.from_dict(early_neoplasia_and_cancer, orient='index')
early_neoplasia_and_cancer_df.columns = ['early_neoplasia_and_cancer_p-value']

total_comparasion = normal_and_early_neoplasia_df.join(early_neoplasia_and_cancer_df)
total_comparasion.head()

,normal_and_early_neoplasia_p-value,early_neoplasia_and_cancer_p-value
LOC643837,0.690766,0.413735
LOC100130417,0.000032,0.653429
SAMD11,0.060273,0.079556
NOC2L,0.826429,0.287581
KLHL17,0.049876,0.463292


In [38]:
total_comparasion[(total_comparasion['normal_and_early_neoplasia_p-value'] < 0.05)].shape

(1575, 9)

In [56]:
write_answer('1.1', '1575')

In [39]:
total_comparasion[(total_comparasion['early_neoplasia_and_cancer_p-value'] < 0.05)].shape

(3490, 9)

In [57]:
write_answer('1.2', '3490')

### Часть 2: поправка методом Холма

In [22]:
import statsmodels.stats.multitest as smm

In [23]:
total_comparasion['normal_mean_expression'] = normal_group.mean()
total_comparasion['early_neoplasia_mean_expression'] = early_neoplasia_group.mean()
total_comparasion['cancer_mean_expression'] = cancer_group.mean()

In [24]:
def abs_fold_change(c, t):
    if t > c:
        return t/c
    else:
        return c/t

In [28]:
total_comparasion['normal_and_neoplasia_fold_change'] = list(map(lambda x, y: abs_fold_change(x, y), 
                                                    total_comparasion.normal_mean_expression,
                                                    total_comparasion.early_neoplasia_mean_expression
                                                   ))
total_comparasion['early_neoplasia_and_cancer_fold_change'] = list(map(lambda x, y: abs_fold_change(x, y),
                                                   total_comparasion.early_neoplasia_mean_expression,
                                                   total_comparasion.cancer_mean_expression
                                                  ))

In [29]:
total_comparasion

,normal_and_early_neoplasia_p-value,early_neoplasia_and_cancer_p-value,normal_mean_expression,early_neoplasia_mean_expression,cancer_mean_expression,normal_and_neoplasia_fold_change,early_neoplasia_and_cancer_fold_change
LOC643837,0.690766,0.413735,2.681277,2.510894,2.186060,1.067858,1.148593
LOC100130417,0.000032,0.653429,4.368497,8.721781,8.190456,1.996517,1.064871
SAMD11,0.060273,0.079556,15.159566,18.531325,23.692614,1.222418,1.278517
NOC2L,0.826429,0.287581,15.374351,15.071854,16.468034,1.020070,1.092635
KLHL17,0.049876,0.463292,21.459886,24.152469,25.035813,1.125471,1.036574
...,...,...,...,...,...,...,...
DDX3Y,0.640904,0.659369,1.701654,1.961449,2.292908,1.152672,1.168987
CD24,0.830134,0.330617,17.106405,17.567902,15.752268,1.026978,1.115262
CYorf15B,0.670395,0.542939,1.576004,1.760707,2.157294,1.117197,1.225243
KDM5D,0.793925,0.565753,1.960442,2.151549,2.776173,1.097481,1.290314


In [31]:
total_comparasion['normal_and_neoplasia_rej_ho'] = smm.multipletests(total_comparasion['normal_and_early_neoplasia_p-value'], alpha=0.025, method='h')[0]
total_comparasion['early_neoplasia_and_cancer_rej_ho'] = smm.multipletests(total_comparasion['early_neoplasia_and_cancer_p-value'], alpha=0.025, method='h')[0]

In [32]:
total_comparasion

,normal_and_early_neoplasia_p-value,early_neoplasia_and_cancer_p-value,normal_mean_expression,early_neoplasia_mean_expression,cancer_mean_expression,normal_and_neoplasia_fold_change,early_neoplasia_and_cancer_fold_change,normal_and_neoplasia_rej_ho,early_neoplasia_and_cancer_rej_ho
LOC643837,0.690766,0.413735,2.681277,2.510894,2.186060,1.067858,1.148593,False,False
LOC100130417,0.000032,0.653429,4.368497,8.721781,8.190456,1.996517,1.064871,False,False
SAMD11,0.060273,0.079556,15.159566,18.531325,23.692614,1.222418,1.278517,False,False
NOC2L,0.826429,0.287581,15.374351,15.071854,16.468034,1.020070,1.092635,False,False
KLHL17,0.049876,0.463292,21.459886,24.152469,25.035813,1.125471,1.036574,False,False
...,...,...,...,...,...,...,...,...,...
DDX3Y,0.640904,0.659369,1.701654,1.961449,2.292908,1.152672,1.168987,False,False
CD24,0.830134,0.330617,17.106405,17.567902,15.752268,1.026978,1.115262,False,False
CYorf15B,0.670395,0.542939,1.576004,1.760707,2.157294,1.117197,1.225243,False,False
KDM5D,0.793925,0.565753,1.960442,2.151549,2.776173,1.097481,1.290314,False,False


In [35]:
total_comparasion[(total_comparasion.normal_and_neoplasia_rej_ho) & (total_comparasion.normal_and_neoplasia_fold_change > 1.5)].shape

(2, 9)

In [58]:
write_answer('2.1', '2')

In [36]:
total_comparasion[(total_comparasion.early_neoplasia_and_cancer_rej_ho) & (total_comparasion.early_neoplasia_and_cancer_fold_change > 1.5)].shape

(77, 9)

In [59]:
write_answer('2.2', '77')

### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [47]:
total_comparasion['normal_and_neoplasia_rej_bh'] = smm.multipletests(total_comparasion['normal_and_early_neoplasia_p-value'], alpha=0.025, method='fdr_i')[0]
total_comparasion['early_neoplasia_and_cancer_rej_bh'] = smm.multipletests(total_comparasion['early_neoplasia_and_cancer_p-value'], alpha=0.025, method='fdr_i')[0]

In [48]:
total_comparasion.normal_and_neoplasia_rej_bh.value_counts()

False    15744
True         4
Name: normal_and_neoplasia_rej_bh, dtype: int64

In [52]:
total_comparasion[(total_comparasion.normal_and_neoplasia_rej_bh) & (total_comparasion.normal_and_neoplasia_fold_change > 1.5)].shape[0]

4

In [60]:
write_answer('3.1', '4')

In [53]:
total_comparasion[(total_comparasion.early_neoplasia_and_cancer_rej_bh) & (total_comparasion.early_neoplasia_and_cancer_fold_change > 1.5)].shape[0]

524

In [61]:
write_answer('3.2', '524')